In [57]:
optim = require 'optim'
fullset = torch.load('imgdata.dat')

In [58]:
shuffle = torch.randperm(fullset.size)
shuffleset = fullset
for i=1, fullset.size do
    shuffleset.data[i] = fullset.data[shuffle[i]]
    shuffleset.label[i] = fullset.label[shuffle[i]]
    end
fullset = shuffleset

In [59]:
trainset = {
    size = 8503,
    data = fullset.data[{{1,8503}}]:double(),
    label = fullset.label[{{1,8503}}]
}

In [60]:
validationset = {
    size = 2000,
    data = fullset.data[{{8504,10503}}]:double(),
    label = fullset.label[{{8504,10503}}]
}

In [61]:
testset = {
    size = 2000,
    data = fullset.data[{{10504,12503}}]:double(),
    label = fullset.label[{{10504,12503}}]
}

In [62]:
nn = require 'nn'
model = nn.Sequential()
model:add(nn.Reshape(2500))

In [63]:
model:add(nn.Linear(2500,120))
model:add(nn.Tanh())
model:add(nn.Linear(120,62))
model:add(nn.LogSoftMax())

In [64]:
criterion = nn.ClassNLLCriterion()

In [65]:
sgd_params = {
   learningRate = 1e-1,
   learningRateDecay = 1e-4,
   weightDecay = 1e-2,
   momentum = 1e-4
}

In [66]:
x, dl_dx = model:getParameters()

In [67]:
step = function(batch_size)
    local current_loss = 0
    local count = 0
    batch_size = batch_size or 200
    for t = 1, trainset.size, batch_size do
        local size = math.min(t + batch_size -1, trainset.size) - t
        local inputs = torch.Tensor(size, 50, 50)
        local targets = torch.Tensor(size)
        for i = 1,size do
            local input = trainset.data[i+t]
            local target = trainset.label[i+t]
            -- if target == 0 then target = 10 end
            inputs[i] = input
            targets[i] = target
        end
        
        
        local feval = function(x_new)
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero()

            -- perform mini-batch gradient descent
            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        current_loss = current_loss + fs[1]
    end

    -- normalize loss
    return current_loss / count
end

In [68]:
eval = function(dataset, batch_size)
    local count = 0
    batch_size = batch_size or 200
    
    for i = 1,dataset.size,batch_size do
        local size = math.min(i + batch_size - 1, dataset.size) - i
        local inputs = dataset.data[{{i,i+size-1}}]
        local targets = dataset.label[{{i,i+size-1}}]:long()
        local outputs = model:forward(inputs)
        local _, indices = torch.max(outputs, 2)
        indices:add(-1)
        local guessed_right = indices:eq(targets):sum()
        count = count + guessed_right
    end

    return count / dataset.size
end

In [69]:
max_iters = 100

In [ ]:
do
    local last_accuracy = 0
    local decreasing = 0
    local threshold = 1 -- how many deacreasing epochs we allow
    for i = 1,max_iters do
        local loss = step()
        print(string.format('Epoch: %d Current loss: %4f', i, loss))
        local accuracy = eval(validationset)
        print(string.format('Accuracy on the validation set: %4f', accuracy))
        if accuracy < last_accuracy then
            if decreasing > threshold then break end
            decreasing = decreasing + 1
        else
            decreasing = 0
        end
        last_accuracy = accuracy
    end
end

Epoch: 1 Current loss: 3.837704	


Accuracy on the validation set: 0.060500	


Epoch: 2 Current loss: 3.786248	


Accuracy on the validation set: 0.060500	


Epoch: 3 Current loss: 3.784971	


...

Accuracy on the validation set: 0.060500	


Epoch: 33 Current loss: 3.782757	


Accuracy on the validation set: 0.060500	


Epoch: 34 Current loss: 3.782696	


Accuracy on the validation set: 0.060500	
